In [1]:
import yaml
import pandas as pd
from datetime import datetime, timedelta

In [2]:
with open ("config.yaml", "r") as config:
    raw = yaml.safe_load(config)['rawsleepdata']
raw    

'data/raw_bed_sleep-state.csv'

In [3]:
df = pd.read_csv(raw)
df['start'] = pd.to_datetime(df['start'], utc=False)
df.sort_values(by=['start'])
df.drop_duplicates(subset="start", keep = 'first', inplace = True)

In [4]:

df['value'] = df['value'].apply(''.join).str.replace('[^\d\s]+', '')
data = []

for index, row in df.iterrows():
    time = row['start']
    sleeptype = 0
    count = 0
    for value_index, value in enumerate(row['value']):
        if int(value) == int(sleeptype):
            count += 1
        else:
            data.append([time, (count*60), sleeptype])
            time = time + timedelta(seconds=(count*60))
            count = 0
            sleeptype = value
            count +=1
           
        if count == len(row['value']):
            data.append([time, (count*60), sleeptype])
            continue
        if (value_index+1) == len(row['value']):
            data.append([time, (count*60), sleeptype])
            
df = pd.DataFrame(data)

/tmp/ipykernel_2285934/2350005278.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['value'] = df['value'].apply(''.join).str.replace('[^\d\s]+', '')


In [5]:
df.columns = ['start', 'duration(s)', 'sleeptype']
df = df[df['duration(s)'] != 0]

In [6]:
df.dtypes

start          datetime64[ns, pytz.FixedOffset(60)]
duration(s)                                   int64
sleeptype                                    object
dtype: object

In [7]:
df.loc[df["sleeptype"] == 0, "sleeptype_word"] = 'awake'
df.loc[df["sleeptype"] == '0', "sleeptype_word"] = 'awake'
df.loc[df["sleeptype"] == '1', "sleeptype_word"] = 'light'
df.loc[df["sleeptype"] == '2', "sleeptype_word"] = 'deep'
df.loc[df["sleeptype"] == '3', "sleeptype_word"] = 'rem'

In [8]:
df.loc[df["sleeptype_word"] == 'awake', "sleeptype"] = '3'
df.loc[df["sleeptype_word"] == 'light', "sleeptype"] = '2'
df.loc[df["sleeptype_word"] == 'deep', "sleeptype"] = '1'
df.loc[df["sleeptype_word"] == 'rem', "sleeptype"] = '0'

In [9]:
df.sort_values(by=['start'], inplace=True)

In [10]:
df['start'] = pd.to_datetime(df['start'], format="%m/%d/%Y %I:%M:%S")

In [11]:
df['hour'] = df['start'].dt.hour

In [12]:
df['day_of_sleep'] = 0

In [13]:
df

,start,duration(s),sleeptype,sleeptype_word,hour,day_of_sleep
675,2022-11-21 23:05:00+01:00,3360,3,awake,23,0
677,2022-11-22 00:01:00+01:00,1020,2,light,0,0
678,2022-11-22 00:18:00+01:00,1200,1,deep,0,0
679,2022-11-22 00:38:00+01:00,840,2,light,0,0
680,2022-11-22 00:52:00+01:00,1500,1,deep,0,0
...,...,...,...,...,...,...
18,2022-12-19 06:35:00+01:00,2880,2,light,6,0
19,2022-12-19 07:23:00+01:00,720,3,awake,7,0
20,2022-12-19 07:35:00+01:00,480,2,light,7,0
21,2022-12-19 07:43:00+01:00,60,0,rem,7,0


In [14]:
count = 0
for index, row in df.iterrows():
    df.loc[df['start'] ==row['start'], 'day_of_sleep'] = count
    if (df.loc[(df['hour'] <12) & (df['start'].dt.day == row['start'].day)]['start'].max() == row['start']):
        count +=1
    
        

In [15]:
df.pop('hour')

675    23
677     0
678     0
679     0
680     0
       ..
18      6
19      7
20      7
21      7
22      7
Name: hour, Length: 647, dtype: int64

In [16]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import FixedTicker, DatetimeTickFormatter
output_notebook()

Loading BokehJS ...

In [17]:
df[df['day_of_sleep'] == 1]

,start,duration(s),sleeptype,sleeptype_word,day_of_sleep
642,2022-11-23 00:34:00+01:00,3900,3,awake,1
643,2022-11-23 01:39:00+01:00,180,2,light,1
644,2022-11-23 01:42:00+01:00,2400,1,deep,1
645,2022-11-23 02:22:00+01:00,600,2,light,1
646,2022-11-23 02:32:00+01:00,600,1,deep,1
647,2022-11-23 02:42:00+01:00,720,0,rem,1
648,2022-11-23 02:54:00+01:00,1140,2,light,1
649,2022-11-23 03:13:00+01:00,960,1,deep,1
650,2022-11-23 03:29:00+01:00,300,2,light,1
652,2022-11-23 03:34:00+01:00,420,2,light,1


In [18]:
def make_plot(day=1):
    p = figure(x_axis_type='datetime')
    p.step(x=df[df['day_of_sleep'] == day]['start'], y=df[df['day_of_sleep'] == 1]['sleeptype'], mode="after")
    p.yaxis[0].ticker=FixedTicker(ticks=[0, 1, 2, 3])
    p.yaxis.major_label_overrides = {0:'Rem', 1:'Deep', 2:'Light', 3:'Awake'}
    p.xaxis.formatter=DatetimeTickFormatter(hours= "%H:%Mh")
    return p

In [19]:
import panel as pn
pn.extension()

In [20]:
interact = pn.interact(make_plot, day=range(0, df['day_of_sleep'].max()+1))

In [21]:
interact

Column
    [0] Column
        [0] DiscreteSlider(formatter='%d', name='day', options=[0, 1, 2, 3, 4, ...], value=1)
    [1] Row
        [0] Bokeh(Figure, name='interactive00101')